In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

data_set_number = 22
data_folder_path = "/content/drive/MyDrive/Deb"
output_folder_path = f"{data_folder_path}/output_csv_folders_tail"

# Check if Data is present
if not os.path.exists(data_folder_path):
  raise FileNotFoundError(f"The path {data_folder_path} does not exist.")
else:
  print("Data is present")

# Create the folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

Data is present


In [ ]:
! pip install torch
! pip install git+https://github.com/huggingface/transformers
! pip install git+https://github.com/huggingface/accelerate
! pip install qwen-vl-utils
!pip install Pillow
!pip install pandas

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cecoxi8e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cecoxi8e
  Resolved https://github.com/huggingface/transformers to commit 78b2929c0554b79e0489b451ce4ece14d265ead2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9786808 sha256=b1f94d966bd5d1c6cf0bdec5e83f214a6be371b8c6fc0b52fa4fb88de9792616
  Stored in directory: /tmp/pip-ephem-wheel-cache-plx1d1gf/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Cloning https://github.com/huggingface

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype=torch.float32, device_map="auto"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [ ]:
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
from PIL import Image, ImageOps
import requests
from io import BytesIO
import pandas as pd

df = pd.read_csv(f"{data_folder_path}/test_part_tail_{data_set_number}.csv")

df.head()

,index,image_link,group_id,entity_name
0,107576,https://m.media-amazon.com/images/I/61FrLYvhle...,849273,height
1,107577,https://m.media-amazon.com/images/I/61FrV3YhPd...,903603,height
2,107578,https://m.media-amazon.com/images/I/61FrV3YhPd...,903603,depth
3,107579,https://m.media-amazon.com/images/I/61FrWdRapp...,483370,width
4,107580,https://m.media-amazon.com/images/I/61FrWdRapp...,483370,height


Regex Block

In [ ]:
import numpy as np

output_df = df.drop(["image_link", "group_id", "entity_name"], axis=1)

# Add a new column 'output' and initialize it with an empty string
output_df["prediction"] = np.nan

# Display the first few rows of the DataFrame
output_df.head()

,index,prediction
0,107576,NaN
1,107577,NaN
2,107578,NaN
3,107579,NaN
4,107580,NaN


In [ ]:
output_df.to_csv(f"{output_folder_path}/output_csv_tail_{data_set_number}_2.csv", index=False)
check = pd.read_csv(f"{output_folder_path}/output_csv_tail_{data_set_number}_2.csv")
check.head()

,index,prediction
0,107576,NaN
1,107577,NaN
2,107578,NaN
3,107579,NaN
4,107580,NaN


In [ ]:
check.shape

(2500, 2)

In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

In [ ]:
# # Replace with your image URL
# image_url = df.iloc[83]['image_link']

# # Download the image
# response = requests.get(image_url)
# img = Image.open(BytesIO(response.content)).convert("RGB")
# display(img)

In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

In [ ]:
import re

def convert_text(received_text):
    # Define the expanded entity unit map
    expanded_entity_unit_map = {
        'width': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'depth': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'height': {
            'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
            'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
            'inch': ['in', 'inch', 'inches', 'IN'],
            'metre': ['m', 'meter', 'metre', 'M'],
            'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
            'yard': ['yd', 'yard', 'Yard', 'YD']
        },
        'item_weight': {
            'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
            'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
            'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
            'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
            'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
            'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
            'ton': ['t', 'ton', 'tons', 'T']
        },
        'maximum_weight_recommendation': {
            'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
            'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
            'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
            'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
            'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
            'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
            'ton': ['t', 'ton', 'tons', 'T']
        },
        'voltage': {
            'kilovolt': ['kV', 'kilovolt', 'kilovolts', 'KV'],
            'millivolt': ['mV', 'millivolt', 'millivolts', 'MV'],
            'volt': ['V', 'volt', 'volts']
        },
        'wattage': {
            'kilowatt': ['kW', 'kilowatt', 'kilowatts', 'KW'],
            'watt': ['W', 'watt', 'watts', 'WATT']
        },
        'item_volume': {
            'centilitre': ['cl', 'centilitre', 'centiliters', 'CL'],
            'cubic foot': ['cu ft', 'cubic foot', 'cubic feet', 'CF'],
            'cubic inch': ['cu in', 'cubic inch', 'cubic inches', 'CI'],
            'cup': ['cup', 'cups', 'C'],
            'decilitre': ['dl', 'decilitre', 'deciliters', 'DL'],
            'fluid ounce': ['fl oz', 'fluid ounce', 'fluid ounces', 'FLOZ'],
            'gallon': ['gal', 'gallon', 'gallons', 'GAL'],
            'imperial gallon': ['imp gal', 'imperial gallon', 'imperial gallons'],
            'litre': ['l', 'litre', 'liter', 'litres', 'L'],
            'microlitre': ['μl', 'microlitre', 'microliters', 'MICROL'],
            'millilitre': ['ml', 'millilitre', 'milliliter', 'milliliters', 'ML'],
            'pint': ['pt', 'pint', 'pints', 'PT'],
            'quart': ['qt', 'quart', 'quarts', 'QT']
        }
    }
    # Remove curly braces and extra spaces
    received_text = received_text.strip('{} ').strip()

    # Extract value and unit using regex
    match = re.match(r'(\d+(\.\d+)?)\s*(\w+)', received_text)
    if not match:
        return ""

    value, unit = match.group(1), match.group(3)

    # Find the correct unit from the expanded_entity_unit_map
    for entity, units in expanded_entity_unit_map.items():
        for key, aliases in units.items():
            if unit in aliases:
                return f"{value} {key}"

    return ""

In [ ]:
import pandas as pd
import requests
from PIL import Image, ImageOps
from io import BytesIO
import torch

# Constants
TIMEOUT_DOWNLOAD = 10  # seconds
TIMEOUT_INFERENCE = 10  # seconds

# Function to download image with timeout and convert to grayscale
def download_image(url, timeout=TIMEOUT_DOWNLOAD, grayscale=False):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))

        if grayscale:
            image = ImageOps.grayscale(image)

        return image
    except requests.exceptions.RequestException as e:
        print(f"Failed to download image from {url}: {e}")
        return None

# Function to prepare inputs and run inference with timeout
def run_inference(image, entity_name, possible_units, timeout=TIMEOUT_INFERENCE):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {
                    "type": "text",
                    "text": f"""Can you tell me the {entity_name} of this product given in the image. Just return the
                     answer strictly in the format - {{value unit}} for example {{10.0 kilogram}}. Don't add anything
                     else in the response. Here are all the {{'possible_units': {possible_units}}}, In the output the
                     spelling of the unit must exactly match with one of the units given in the possible_units map, if not then convert it
                     to match one of the units in possible_units , the
                      value of which you detected from the image. If you can't find it, just return an empty string""",
                },
            ],
        }
    ]

    # Prepare inputs
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    try:
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            return output_text[0]
    except Exception as e:
        print(f"Inference failed: {e}")
        return None

# Main function to process the DataFrame
def process_dataframe(test_df):
    for index, row in test_df.iterrows():
        image_url = row["image_link"]
        entity_name = row["entity_name"]
        corresponding_index = row["index"]

        # Check if entity_name is valid
        if entity_name not in entity_unit_map:
            print(f"Invalid entity name: {entity_name}")
            output_df.at[index, "output"] = np.nan
            continue
        possible_units = entity_unit_map[entity_name]
        # Download image
        image = download_image(image_url, grayscale=False)
        if image is None:
            output_df.at[index, "output"] = np.nan
            continue

        # Run inference with timeout
        output = convert_text(run_inference(image, entity_name, possible_units, timeout=TIMEOUT_INFERENCE))
        print(f"Completed index {corresponding_index} - total {index} - output: {output}")
        output_df.at[index, "output"] = output

        # Save to CSV every 10th interval
        if index % 10 == 0:
            try:
                output_df.to_csv(f"{output_folder_path}/output_csv_tail_{data_set_number}_2.csv", index=False)
            except Exception as e:
                print(f"Error saving to CSV: {e}")
    return output_df

In [ ]:
df.shape

(2500, 4)

In [ ]:
process_dataframe(df)
x = pd.read_csv(f"{output_folder_path}/output_csv_tail_{data_set_number}_2.csv")
x.head(15)

Completed index 107576 - total 0 - output: 20.0 foot
Completed index 107577 - total 1 - output: 22 millimetre
Completed index 107578 - total 2 - output: 22 millimetre
Completed index 107579 - total 3 - output: 25.0 centimetre
Completed index 107580 - total 4 - output: 25.0 centimetre
Completed index 107581 - total 5 - output: 25.0 centimetre
Completed index 107582 - total 6 - output: 152.4 millimetre
Completed index 107583 - total 7 - output: 33 millimetre
Completed index 107584 - total 8 - output: 33 millimetre
Completed index 107585 - total 9 - output: 13 centimetre
Completed index 107586 - total 10 - output: 0.8 inch
Completed index 107587 - total 11 - output: 10.0 inch
Completed index 107588 - total 12 - output: 3.5 inch
Completed index 107589 - total 13 - output: 5.0 inch
Completed index 107590 - total 14 - output: 45.0 centimetre
Completed index 107591 - total 15 - output: 7.08 inch
Completed index 107592 - total 16 - output: 7.5 centimetre
Completed index 107593 - total 17 - out

KeyboardInterrupt: 

In [ ]:
x.sample(15, random_state = 42)

# Spliting the csv


In [ ]:
# import os
# import pandas as pd

# # Path to your original CSV file in Google Drive
# csv_path = '/content/drive/My Drive/AMAZON/test.csv'

# # Directory to save the split files
# output_dir = '/content/drive/My Drive/AMAZON/final_test/'

# # Create the output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)


In [ ]:
# # Load the CSV file
# df = pd.read_csv(csv_path)

# # Number of rows per split
# chunk_size = 5000

# # Loop over the data and create separate CSV files
# for i in range(0, len(df), chunk_size):
#     # Define the start and end of the chunk
#     chunk = df[i:i + chunk_size]

#     # Create the file name for each chunk
#     chunk_file = os.path.join(output_dir, f'test_part_{i//chunk_size + 1}.csv')

#     # Save the chunk as a CSV file
#     chunk.to_csv(chunk_file, index=False)

#     print(f'Saved {chunk_file}')


In [ ]:
# import pandas as pd

# # Replace with the actual path to one of the split files
# split_file_path = '/content/drive/MyDrive/AMAZON/final_test/test_part_1.csv'

# # Load and display the first few rows of the file
# df_split = pd.read_csv(split_file_path)
# print(df_split.tail(2))  # Show the first 5 rows


In [ ]:
# import pandas as pd

# # Replace with the actual path to one of the split files
# split_file_path = '/content/drive/MyDrive/AMAZON/final_test/test_part_2.csv'

# # Load and display the first few rows of the file
# df_split = pd.read_csv(split_file_path)
# print(df_split.head(2))  # Show the first 5 rows


# Getting result from each csv

In [ ]:
# import shutil

# # Define the folder path you want to zip
# folder_to_zip = '/content/drive/MyDrive/AMAZON/final_test'

# # Output path for the zip file
# output_zip = '/content/final_test_split.zip'

# # Zip the folder
# shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_to_zip)


In [ ]:
# from google.colab import files

# # Download the zip file
# files.download(output_zip)


In [ ]:
# from PIL import Image, ImageOps
# import requests
# from io import BytesIO
# import pandas as pd

# df = pd.read_csv('/content/drive/MyDrive/AMAZON/final_test/test_part_8.csv')

# df.shape


In [ ]:
# # Replace with your image URL
# image_url = df.iloc[83]['image_link']

# # Download the image
# response = requests.get(image_url)
# img = Image.open(BytesIO(response.content)).convert("RGB")
# display(img)

In [ ]:
# import re

# def convert_text(received_text):
#     # Define the expanded entity unit map
#     expanded_entity_unit_map = {
#         'width': {
#             'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
#             'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
#             'inch': ['in', 'inch', 'inches', 'IN'],
#             'metre': ['m', 'meter', 'metre', 'M'],
#             'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
#             'yard': ['yd', 'yard', 'Yard', 'YD']
#         },
#         'depth': {
#             'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
#             'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
#             'inch': ['in', 'inch', 'inches', 'IN'],
#             'metre': ['m', 'meter', 'metre', 'M'],
#             'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
#             'yard': ['yd', 'yard', 'Yard', 'YD']
#         },
#         'height': {
#             'centimetre': ['cm', 'centimeter', 'centimetre', 'centi', 'CM'],
#             'foot': ['ft', 'foot', 'FT', 'Feet', 'feet'],
#             'inch': ['in', 'inch', 'inches', 'IN'],
#             'metre': ['m', 'meter', 'metre', 'M'],
#             'millimetre': ['mm', 'millimeter', 'millimetre', 'MM'],
#             'yard': ['yd', 'yard', 'Yard', 'YD']
#         },
#         'item_weight': {
#             'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
#             'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
#             'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
#             'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
#             'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
#             'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
#             'ton': ['t', 'ton', 'tons', 'T']
#         },
#         'maximum_weight_recommendation': {
#             'gram': ['g', 'gm', 'gms', 'grams', 'gram', 'G', 'Gr', 'GRM', 'grms', 'grm'],
#             'kilogram': ['kg', 'kgs', 'kilogram', 'kilograms', 'KG', 'kilo'],
#             'microgram': ['mcg', 'μg', 'microgram', 'micrograms', 'MG'],
#             'milligram': ['mg', 'milligram', 'milligrams', 'MG'],
#             'ounce': ['oz', 'ounce', 'ounces', 'OZ'],
#             'pound': ['lb', 'lbs', 'pound', 'pounds', 'LB'],
#             'ton': ['t', 'ton', 'tons', 'T']
#         },
#         'voltage': {
#             'kilovolt': ['kV', 'kilovolt', 'kilovolts', 'KV'],
#             'millivolt': ['mV', 'millivolt', 'millivolts', 'MV'],
#             'volt': ['V', 'volt', 'volts']
#         },
#         'wattage': {
#             'kilowatt': ['kW', 'kilowatt', 'kilowatts', 'KW'],
#             'watt': ['W', 'watt', 'watts', 'WATT']
#         },
#         'item_volume': {
#             'centilitre': ['cl', 'centilitre', 'centiliters', 'CL'],
#             'cubic foot': ['cu ft', 'cubic foot', 'cubic feet', 'CF'],
#             'cubic inch': ['cu in', 'cubic inch', 'cubic inches', 'CI'],
#             'cup': ['cup', 'cups', 'C'],
#             'decilitre': ['dl', 'decilitre', 'deciliters', 'DL'],
#             'fluid ounce': ['fl oz', 'fluid ounce', 'fluid ounces', 'FLOZ'],
#             'gallon': ['gal', 'gallon', 'gallons', 'GAL'],
#             'imperial gallon': ['imp gal', 'imperial gallon', 'imperial gallons'],
#             'litre': ['l', 'litre', 'liter', 'litres', 'L'],
#             'microlitre': ['μl', 'microlitre', 'microliters', 'MICROL'],
#             'millilitre': ['ml', 'millilitre', 'milliliter', 'milliliters', 'ML'],
#             'pint': ['pt', 'pint', 'pints', 'PT'],
#             'quart': ['qt', 'quart', 'quarts', 'QT']
#         }
#     }
#     # Remove curly braces and extra spaces
#     received_text = received_text.strip('{} ').strip()

#     # Extract value and unit using regex
#     match = re.match(r'(\d+(\.\d+)?)\s*(\w+)', received_text)
#     if not match:
#         return ""

#     value, unit = match.group(1), match.group(3)

#     # Find the correct unit from the expanded_entity_unit_map
#     for entity, units in expanded_entity_unit_map.items():
#         for key, aliases in units.items():
#             if unit in aliases:
#                 return f"{value} {key}"

#     return ""

In [ ]:
# df.head()